In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
DATA_DIR = '/home/HDD6TB/datasets/emotions/ABAW'

In [3]:
import os
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn import svm,metrics,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score,roc_auc_score,average_precision_score
import mord

from collections import defaultdict
import os
import glob
import random
import numpy as np
from tqdm import tqdm
import time
import pickle
import pandas as pd
import random
import csv  
import matplotlib.pyplot as plt

In [4]:
def get_names(id):
    name = ""
    if id>=0 and id<10:
        name = "0000" + str(id)
    elif id>=10 and id<100:
        name = "000" + str(id)
    elif id>=100 and id<1000:
        name = "00" + str(id)
    elif id>=1000 and id<10000:
        name = "0" + str(id)
    else:
        name = str(id)
    return name

In [5]:
compare_filenames=lambda x: int(os.path.splitext(x)[0])

In [6]:
import pathlib
import tensorflow as tf
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.layers import TimeDistributed, GRU, Dense, Dropout, Flatten, LSTM, Activation, MaxPooling2D
from tensorflow.keras.regularizers import l2 as L2_reg
from tensorflow.keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D, Input, Masking, Conv3D, MaxPooling3D, GlobalMaxPool3D
from tensorflow.keras.optimizers import SGD, Adam
import tensorflow.keras.backend as K 

print(tf.__version__)
from tensorflow.compat.v1.keras.backend import set_session 
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(config=config)
set_session(sess)

2.4.1


# Number of frames

In [7]:
data_dir=os.path.join(DATA_DIR,'faces')
d=os.path.join(DATA_DIR,'videos')
video2len={}
for filename in os.listdir(d):
    fn, ext = os.path.splitext(os.path.basename(filename))
    vid=os.path.join(d,filename)
    cap = cv2.VideoCapture(vid)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video2len[fn]=total_frames+1 #FIX ME!!! NEED TO ADD 1 to teh number of frames for consistency with challeng's organizer
    
    dn=os.path.join(data_dir,fn)
    last_img=''
    if os.path.exists(dn):
        images=[img_name for img_name in os.listdir(dn) if img_name.lower().endswith('.jpg')]
        last_img=sorted(images, key=compare_filenames)[-1]
    elif os.path.exists(dn+'_left'):
        images=[img_name for img_name in os.listdir(dn+'_left') if img_name.lower().endswith('.jpg')]
        last_img=sorted(images, key=compare_filenames)[-1]
    print(fn,total_frames,last_img)
        

27-60-1280x720 2932 02933.jpg
video39 10678 10671.jpg
77-30-1280x720 5672 05673.jpg
91-30-1920x1080 10921 10922.jpg
360 3509 03473.jpg
136 4487 04488.jpg
197 1869 01786.jpg
291 1830 01831.jpg
16-30-1920x1080 5473 05474.jpg
video2 2064 02055.jpg
89-30-1080x1920 630 00631.jpg
408 2978 02978.jpg
110-30-270x480 9978 09979.jpg
324 5614 05614.jpg
14-30-1920x1080 5310 05311.jpg
129-24-1280x720 6153 06154.jpg
153 3677 03678.jpg
314 2260 02260.jpg
255 1691 01692.jpg
284 4054 04054.jpg
298 2836 02836.jpg
206 4476 04475.jpg
video84 525 00525.jpg
387 2911 02911.jpg
286 2677 02678.jpg
55-25-1280x720 1505 01480.jpg
336 4344 04344.jpg
video52 10679 10671.jpg
60-30-1920x1080 4564 04564.jpg
120 3591 03592.jpg
267 3841 03841.jpg
121-24-1920x1080 10628 10566.jpg
349 4054 04054.jpg
46-30-484x360 2944 02945.jpg
218 3849 03830.jpg
video55 8419 08409.jpg
368 2929 02929.jpg
9-15-1920x1080 47434 47435.jpg
384 3451 03451.jpg
228 2155 02155.jpg
video3 7801 07802.jpg
106-30-720x1280 1404 01405.jpg
64-24-640x360 4

264 4744 04745.jpg
video30 8217 08214.jpg
341 3096 03098.jpg
215 3409 03378.jpg
94-30-1920x1080 2101 02102.jpg
421 3615 03615.jpg
138 5583 05584.jpg
117 5465 05466.jpg
video42 9226 09217.jpg
6-30-1920x1080 7985 07986.jpg
332 4665 04560.jpg
409 3608 03608.jpg
44-25-426x240 300 00301.jpg
154 3362 03363.jpg
407 3817 03817.jpg
41-24-1280x720 6160 06161.jpg
5-60-1920x1080-3 6333 06334.jpg
158 3031 03032.jpg
43-30-406x720 2873 02874.jpg
video94 696 00696.jpg
392 3550 03550.jpg
video19 784 00775.jpg
183 2769 02770.jpg
video6 5324 05325.jpg
video65 4571 04572.jpg
250 2928 02928.jpg
5-60-1920x1080-2 6213 06214.jpg
3-25-1920x1080 6188 06189.jpg
363 3240 03240.jpg
233 4479 04360.jpg
52-30-1280x720 1924 01863.jpg
299 2336 02337.jpg
135 3591 03593.jpg
329 5086 05087.jpg
video37 9186 09116.jpg
video62 5579 05579.jpg
423 5265 05265.jpg
416 6511 06512.jpg
122 4127 04128.jpg
21-24-1920x1080 5230 05231.jpg
369 4438 04437.jpg
video16 12486 12487.jpg
223 2797 02797.jpg
video48 2542 02534.jpg
134-30-1280x7

## Tensorflow

In [44]:
from tensorflow.keras.applications import mobilenet
from tensorflow.keras.models import load_model,Model

In [45]:
def mobilenet_preprocess_input(x,**kwargs):
    x[..., 0] -= 103.939
    x[..., 1] -= 116.779
    x[..., 2] -= 123.68
    return x

preprocessing_function=mobilenet_preprocess_input

In [46]:
model_name='mymobilenet_7_ft_sgd_model'
num_classes=7

base_model=load_model('../../models/affectnet_emotions/'+model_name+'.h5')
_,w,h,_=base_model.input.shape
print(w,h)

224 224


In [10]:
feature_extractor_model=Model(base_model.input,[base_model.get_layer('global_pooling').output,base_model.get_layer('feats').output,base_model.output])
feature_extractor_model.summary()
print(feature_extractor_model.output)

[<KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'global_pooling')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'feats')>,
 <KerasTensor: shape=(None, 7) dtype=float32 (created by layer 'emotion_preds')>]

In [11]:
weights,bias=base_model.get_layer('emotion_preds').get_weights()
print(weights.shape,bias.shape)

def get_probab(features, logits=False):
    x=np.dot(features,weights)+bias
    if logits:
        return x
    e_x = np.exp(x - np.max(x,axis=0))
    return e_x / e_x.sum(axis=1)[:,None]

(256, 7) (7,)


In [19]:
data_dir=os.path.join(DATA_DIR,'faces')
print(data_dir)
img_names=[]
X_global_features,X_scores=[],[]
imgs=[]
for filename in tqdm(os.listdir(data_dir)):
    frames_dir=os.path.join(data_dir,filename)
    for img_name in os.listdir(frames_dir):
        if img_name.lower().endswith('.jpg'):
            img=cv2.imread(os.path.join(frames_dir,img_name))
            if img.size:
                img_names.append(filename+'/'+img_name)
                img=cv2.resize(img,(w,h))
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                imgs.append(img)
                if len(imgs)>=512:        
                    inp = preprocessing_function(np.array(imgs, dtype=np.float32))
                    global_features,feats,scores=feature_extractor_model.predict(inp)
                    if len(X_scores)==0:
                        X_global_features=global_features
                        X_scores=scores
                    else:
                        X_global_features=np.concatenate((X_global_features,global_features),axis=0)
                        X_scores=np.concatenate((X_scores,scores),axis=0)
                    imgs=[]

if len(imgs)>0:        
    inp = preprocessing_function(np.array(imgs, dtype=np.float32))
    global_features,feats,scores=feature_extractor_model.predict(inp)
    if len(X_scores)==0:
        X_global_features=global_features
        X_scores=scores
    else:
        X_global_features=np.concatenate((X_global_features,global_features),axis=0)
        X_scores=np.concatenate((X_scores,scores),axis=0)

print(len(img_names),X_global_features.shape,X_scores.shape)

  0%|          | 0/564 [00:00<?, ?it/s]

/home/HDD6TB/datasets/emotions/ABAW/faces


100%|██████████| 564/564 [5:04:48<00:00, 32.43s/it]    


2658563 (2658563, 1024) (2658563, 7)


In [30]:
filename2featuresAll={img_name:(global_features,scores) for img_name,global_features,scores in zip(img_names,X_global_features,X_scores)}
print(len(filename2featuresAll))

2658563


## PyTorch

In [31]:
import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

print(f"Torch: {torch.__version__}")
device = 'cuda'
use_cuda = torch.cuda.is_available()
print(use_cuda)

Torch: 1.7.1+cu110
True


In [32]:
if False:
    PATH='enet_b2_8.pt'
    #PATH='enet_b2_7.pt'
    IMG_SIZE=260
else:
    #PATH='enet_b0_7.pt'
    #PATH='enet_b0_8_best_afew.pt'
    PATH='enet_b0_8_best_vgaf.pt'
    IMG_SIZE=224
    
test_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

print(PATH)
feature_extractor_model = torch.load('../../models/affectnet_emotions/'+PATH)

enet_b0_8_best_vgaf.pt


In [33]:
if True:
    classifier_weights=feature_extractor_model.classifier[0].weight.cpu().data.numpy()
    classifier_bias=feature_extractor_model.classifier[0].bias.cpu().data.numpy()
else:
    classifier_weights=feature_extractor_model.classifier.weight.cpu().data.numpy()
    classifier_bias=feature_extractor_model.classifier.bias.cpu().data.numpy()
print(classifier_weights.shape,classifier_weights)
print(classifier_bias.shape,classifier_bias)

(8, 1280) [[-0.00668902  0.11335418  0.04948402 ...  0.06092974 -0.02686705
  -0.2584354 ]
 [-0.05986994 -0.11609969 -0.11716661 ... -0.0897665   0.02036773
   0.11924423]
 [ 0.05512754 -0.0534068  -0.001405   ...  0.08906588 -0.12142781
   0.0115855 ]
 ...
 [-0.060075    0.05815501 -0.03156688 ... -0.0667517   0.09121078
   0.06233402]
 [ 0.08181744  0.1318671   0.25012463 ...  0.03424729 -0.0969213
   0.15267523]
 [ 0.10014281 -0.07073897 -0.13113672 ... -0.11299834  0.12007102
  -0.12579045]]
(8,) [-0.08768775  0.00056133 -0.0919608   0.00036013  0.1872526   0.14050813
 -0.09973471  0.01581638]


In [34]:
feature_extractor_model.classifier=torch.nn.Identity()
feature_extractor_model=feature_extractor_model.to(device)
feature_extractor_model.eval()

EfficientNet(
  (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
    )
    (1): Sequential(
      (0

In [35]:
def get_probab(features, logits=True):
    x=np.dot(features,np.transpose(classifier_weights))+classifier_bias
    if logits:
        return x
    e_x = np.exp(x - np.max(x,axis=0))
    return e_x / e_x.sum(axis=1)[:,None]

In [36]:
print(test_transforms)
data_dir=os.path.join(DATA_DIR,'faces')
#data_dir=os.path.join(DATA_DIR,'cropped_aligned')
print(data_dir)
img_names=[]
X_global_features=[]
imgs=[]
for filename in tqdm(os.listdir(data_dir)):
    frames_dir=os.path.join(data_dir,filename)    
    for img_name in os.listdir(frames_dir):
        if img_name.lower().endswith('.jpg'):
            img = Image.open(os.path.join(frames_dir,img_name))
            img_tensor = test_transforms(img)
            if img.size:
                img_names.append(filename+'/'+img_name)
                imgs.append(img_tensor)
                if len(imgs)>=96: #48: #64: #32:        
                    features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
                    features=features.data.cpu().numpy()
                    
                    if len(X_global_features)==0:
                        X_global_features=features
                    else:
                        X_global_features=np.concatenate((X_global_features,features),axis=0)
                    imgs=[]

if len(imgs)>0:        
    features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
    features=features.data.cpu().numpy()

    if len(X_global_features)==0:
        X_global_features=features
    else:
        X_global_features=np.concatenate((X_global_features,features),axis=0)

    imgs=[]

Compose(
    Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
/home/HDD6TB/datasets/emotions/ABAW/cropped_aligned


In [37]:
X_scores=get_probab(X_global_features)

In [18]:
filename2featuresAll={img_name:(global_features,scores) for img_name,global_features,scores in zip(img_names,X_global_features,X_scores)}
print(len(filename2featuresAll))

2657113


### Save/load features 

In [7]:
import pickle
if False:
    model_name='mymobilenet_7_ft_sgd_model'
    num_classes=7
else:
    num_classes=8
    model_name='enet_b0_8_best_vgaf' #first three challenges
    #model_name='enet2_8' #MTL challenge
    
MODEL2FEATURES=model_name+'.pickle' 

print(MODEL2FEATURES)

enet_b0_8_best_vgaf.pickle


In [41]:
if False:
    with open(MODEL2FEATURES, 'wb') as handle:
        pickle.dump(filename2featuresAll, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
#with open('../../../emotions-multimodal/faces/ABAW/'+MODEL2FEATURES, 'rb') as handle:
with open(MODEL2FEATURES, 'rb') as handle:
    filename2featuresAll=pickle.load(handle)
print(len(filename2featuresAll))

2658822


# Classifiers

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential, load_model,model_from_json
from tensorflow.keras.applications import mobilenet,mobilenet_v2,densenet,inception_resnet_v2,inception_v3,vgg16,resnet_v2,resnet
import efficientnet.tfkeras as enet
from tensorflow.keras.layers import Flatten, Dense, Dropout,GlobalAveragePooling2D,Activation, Conv2D, Reshape,DepthwiseConv2D,Input
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, Callback, CSVLogger, EarlyStopping
from tensorflow.keras.metrics import top_k_categorical_accuracy
import numpy as np
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
from copy import deepcopy
class SaveBestModel(tf.keras.callbacks.Callback):
    def __init__(self, save_best_metric='val_loss', this_max=False):
        self.save_best_metric = save_best_metric
        self.max = this_max
        if this_max:
            self.best = float('-inf')
        else:
            self.best = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        metric_value = logs[self.save_best_metric]
        if self.max:
            if metric_value > self.best:
                self.best = metric_value
                self.best_model_weights = deepcopy(self.model.get_weights())

        else:
            if metric_value < self.best:
                self.best = metric_value
                self.best_model_weights = deepcopy(self.model.get_weights())

In [11]:
def one_hot_transfer(label,class_num):
    return np.eye(class_num)[label]

def metric_for_Exp(gt,pred,class_num=8):
    acc = accuracy_score(gt,pred)
    
    # compute_F1
    gt = one_hot_transfer(gt,class_num)
    pred = one_hot_transfer(pred,class_num)
    F1 = []
    for i in range(class_num):
        gt_ = gt[:,i]
        pred_ = pred[:,i]
        F1.append(f1_score(gt_.flatten(), pred_))
    F1_mean = np.mean(F1)
    return F1_mean,acc,F1

In [12]:
def CCC_score(x, y):
    vx = x - np.mean(x)
    vy = y - np.mean(y)
    rho = np.sum(vx * vy) / (np.sqrt(np.sum(vx**2)) * np.sqrt(np.sum(vy**2)))
    x_m = np.mean(x)
    y_m = np.mean(y)
    x_s = np.std(x)
    y_s = np.std(y)
    ccc = 2*rho*x_s*y_s/(x_s**2 + y_s**2 + (x_m - y_m)**2)
    return ccc

def metric_for_VA(gt_V,gt_A,pred_V,pred_A):
    ccc_V,ccc_A = CCC_score(gt_V,pred_V),CCC_score(gt_A,pred_A)
    return ccc_V,ccc_A, 0.5*(ccc_V+ccc_A)

def CCC_numpy(y_true, y_pred):
    '''Reference numpy implementation of Lin's Concordance correlation coefficient'''
    
    # covariance between y_true and y_pred
    s_xy = np.cov([y_true, y_pred])[0,1]
    # means
    x_m = np.mean(y_true)
    y_m = np.mean(y_pred)
    # variances
    s_x_sq = np.var(y_true)
    s_y_sq = np.var(y_pred)
    
    # condordance correlation coefficient
    ccc = (2.0*s_xy) / (s_x_sq + s_y_sq + (x_m-y_m)**2)
    
    return ccc

def CCC(y_true, y_pred):
    '''Lin's Concordance correlation coefficient: https://en.wikipedia.org/wiki/Concordance_correlation_coefficient
    
    The concordance correlation coefficient is the correlation between two variables that fall on the 45 degree line through the origin.
    
    It is a product of
    - precision (Pearson correlation coefficient) and
    - accuracy (closeness to 45 degree line)

    Interpretation:
    - `rho_c =  1` : perfect agreement
    - `rho_c =  0` : no agreement
    - `rho_c = -1` : perfect disagreement 
    
    Args: 
    - y_true: ground truth
    - y_pred: predicted values
    
    Returns:
    - concordance correlation coefficient (float)
    '''
    
    # covariance between y_true and y_pred
    s_xy = K.mean((y_true - K.mean(y_true)) * (y_pred - K.mean(y_pred)))
    # means
    x_m = K.mean(y_true)
    y_m = K.mean(y_pred)
    # variances
    s_x_sq = K.var(y_true)
    s_y_sq = K.var(y_pred)
    
    # condordance correlation coefficient
    ccc = (2.0*s_xy) / (s_x_sq + s_y_sq + (x_m-y_m)**2+K.epsilon())
    return ccc

def CCC_VA(y_true, y_pred):
    return 1-0.5*(CCC(y_true[:,0], y_pred[:,0])+CCC(y_true[:,1], y_pred[:,1]))


In [13]:
import math
def f1_score_max_for_AU_one_class(gt, pred, thresh,type=0):
    gt = gt[:,type]
    pred = pred[:,type]
    P = []
    R = []
    ACC = []
    F1 = []
    for i in thresh:
        new_pred = ((pred >= i) * 1).flatten()
        P.append(precision_score(gt.flatten(), new_pred))
        R.append(recall_score(gt.flatten(), new_pred))
        ACC.append(accuracy_score(gt.flatten(), new_pred))
        F1.append(f1_score(gt.flatten(), new_pred))

    F1_MAX = max(F1)
    if F1_MAX < 0 or math.isnan(F1_MAX):
        F1_MAX = 0
        F1_THRESH = 0
        accuracy = 0
    else:
        idx_thresh = np.argmax(F1)
        F1_THRESH = thresh[idx_thresh]
        accuracy = ACC[idx_thresh]
    return F1,F1_MAX,F1_THRESH,accuracy

def f1_score_max(gt, pred, thresh,c=12):
    F1_s = []
    F1_t = []
    ACC = []
    from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
    for i in range(c):
        F1, F1_MAX, F1_THRESH,acc = f1_score_max_for_AU_one_class(gt,pred,thresh,i)
        F1_s.append(F1_MAX)
        F1_t.append(F1_THRESH)
        ACC.append(acc)
    F1_s=np.array(F1_s)
    F1_t=np.array(F1_t)
    ACC=np.array(ACC)
    return F1_s.mean(),F1_t.mean(),F1_s,F1_t,ACC

## Expr 

In [14]:
def get_image2Expr(dirname):
    dirpath=os.path.join(DATA_DIR,'Third ABAW Annotations/EXPR_Classification_Challenge/',dirname)
    num_missed=0
    X,y=[],[]
    minConstantFrames,numConstant=100000,0
    for filename in os.listdir(dirpath):
        fn, ext = os.path.splitext(os.path.basename(filename))
        if ext.lower()=='.txt':
            with open(os.path.join(dirpath,filename)) as f:
                lines = f.read().splitlines()
                prev_val=None
                for i,line in enumerate(lines):
                    if i>0:
                        expression=int(line)
                        if expression>=0:
                            if prev_val is None:
                                prev_val=expression
                                numConstant=1
                            elif prev_val==expression:
                                numConstant+=1
                            else:
                                if numConstant<minConstantFrames:
                                    minConstantFrames=numConstant
                                prev_val=expression
                                numConstant=1
                            imagename=fn+'/'+get_names(i)+'.jpg'
                            if imagename in filename2featuresAll:
                                X.append(filename2featuresAll[imagename][0])
                                y.append(expression)
                            else:
                                num_missed+=1
    X=np.array(X)
    y=np.array(y)
    print(X.shape,y.shape,num_missed,minConstantFrames)
    return X,y

X_train,y_train=get_image2Expr('Train_Set')
X_val,y_val=get_image2Expr('Validation_Set')

(585317, 1280) (585317,) 12044 1
(280532, 1280) (280532,) 3698 3


In [15]:
def print_expr():
    y_val_preds=mlpModel.predict(X_val)
    y_pred=np.argmax(y_val_preds,axis=1)
    print('Acc:',(y_pred==y_val).mean(), 'F1:',f1_score(y_true=y_val,y_pred=y_pred, average="macro"))
    print(metric_for_Exp(y_val,y_pred))

In [16]:
(unique, counts) = np.unique(y_train, return_counts=True)
num_classes=len(unique)
cw=1/counts
cw/=cw.min()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
print(counts, class_weights, num_classes, unique)

[177198  16573  10771   9080  95463  78751  31615 165866] {0: 1.0, 1: 10.691968865021419, 2: 16.451397270448425, 3: 19.515198237885464, 4: 1.856195594104522, 5: 2.2501047605744686, 6: 5.6048711054879, 7: 1.0683202102902343} 8 [0 1 2 3 4 5 6 7]


In [17]:
batch_size=256 #128
mlpModel=Sequential()
if False:
    mlpModel.add(Dense(num_classes, input_shape=X_train.shape[1:],activation='softmax',use_bias=True,kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size)))
else:
    mlpModel.add(Dense(128, input_shape=X_train.shape[1:],activation='relu')) #256
    mlpModel.add(Dense(num_classes,activation='softmax'))

In [38]:
mlpModel.compile(optimizer=Adam(lr=1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mlpModel.summary()

save_best_model = SaveBestModel('val_accuracy',True)
mlpModel.fit(X_train,y_train, batch_size=batch_size, epochs=10, verbose=1, 
             callbacks=[save_best_model], validation_data=(X_val,y_val),class_weight=class_weights)
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               163968    
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 1032      
Total params: 165,000
Trainable params: 165,000
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2287/2287 [==============================] - 8s 4ms/step - loss: 1.5398 - accuracy: 0.7604 - val_loss: 1.9589 - val_accuracy: 0.4541
Epoch 2/10
2287/2287 [==============================] - 7s 3ms/step - loss: 0.5820 - accuracy: 0.8989 - val_loss: 2.2295 - val_accuracy: 0.4642
Epoch 3/10
2287/2287 [==============================] - 8s 3ms/step - loss: 0.4193 - accuracy: 0.9240 - val_loss: 2.5471 - val_accuracy: 0.4654
Epoch 4/10
2287/2287 [==============================] - 8s 3ms/step - loss: 0.

In [39]:
if True:
    print_expr()
    print('Best weights:')
    mlpModel.set_weights(best_model_weights)
print_expr()

Acc: 0.4298012347967433 F1: 0.3418451036131609
(0.3418451036131609, 0.4298012347967433, [0.5287005218276696, 0.18212996389891697, 0.5914928499863376, 0.018604651162790694, 0.46076099313138263, 0.3311925510325892, 0.22376455670859888, 0.39811474115700163])
Best weights:
Acc: 0.4654121454949881 F1: 0.34060359780179217
(0.34060359780179217, 0.4654121454949881, [0.5619842406043042, 0.1331245105716523, 0.4915926179084074, 0.009731543624161074, 0.4858991788569254, 0.3211159481346253, 0.23595084924606013, 0.4854298934682019])


In [18]:
if False:
    #mlpModel.save_weights('expr_enet0_7.h5') #Acc: 0.48438324326636534 F1: 0.3604978104476117
    mlpModel.save_weights('expr_enet0_vgaf.h5')#Acc: 0.500285172458044 F1: 0.3807067519117502
else:
    #mlpModel.load_weights('../../../emotions-multimodal/faces/ABAW/expr_enet0_vgaf.h5')
    mlpModel.load_weights('expr_enet0_vgaf.h5')
    print_expr()

Acc: 0.500285172458044 F1: 0.3807067519117502
(0.3807067519117502, 0.500285172458044, [0.608643679809698, 0.15128569174948728, 0.5162256615077384, 0.01599360255897641, 0.4778312968664514, 0.4611138100075078, 0.3029224246739377, 0.5116378481202044])


### Smooth validation predictions  

In [19]:
data_dir=os.path.join(DATA_DIR,'faces')
dirpath=os.path.join(DATA_DIR,'Third ABAW Annotations/EXPR_Classification_Challenge/Validation_Set')
test_videos={}
for filename in tqdm(os.listdir(dirpath)):
    fn, ext = os.path.splitext(os.path.basename(filename))
    if ext.lower()=='.txt':
        X,indices,expressions=[],[],[]
        with open(os.path.join(dirpath,filename)) as f:
            lines = f.read().splitlines()
            prev_val=None
            for i,line in enumerate(lines):
                if i>0:
                    imagename=fn+'/'+get_names(i)+'.jpg'
                    if imagename in filename2featuresAll:
                        X.append(filename2featuresAll[imagename][0])
                        indices.append(i)
                        expressions.append(int(line))
        test_videos[fn]=(mlpModel.predict(np.array(X)),indices,np.array(expressions))
print(len(test_videos))

100%|██████████| 70/70 [00:22<00:00,  3.06it/s]

70


In [85]:
hyperparams=[(isMean,delta) for delta in [0,2,7]  for isMean in [1,0] if not (isMean==0 and delta==0)]
total_true=[]
total_preds=[[] for _ in range(len(hyperparams))]
for videoname,(y_pred_expr,indices,expressions) in test_videos.items():
    for i,ind in enumerate(indices):
        if expressions[i]>=0:
            total_true.append(expressions[i])
    cur_ind=0
    preds_proba=[]
    for i in range(indices[-1]):
        if indices[cur_ind]-1==i:
            preds_proba.append(y_pred_expr[cur_ind])
            cur_ind+=1
        else:
            if cur_ind==0:
                preds_proba.append(y_pred_expr[cur_ind])
            else:
                w=(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                pred=w*y_pred_expr[cur_ind-1]+(1-w)*y_pred_expr[cur_ind]
                preds_proba.append(pred)
    
    preds_proba=np.array(preds_proba)
    for hInd,(isMean,delta) in enumerate(hyperparams):
        preds=[]
        for i in range(len(preds_proba)):
            i1=max(i-delta,0)
            if isMean:
                proba=np.mean(preds_proba[i1:i+delta+1],axis=0)
            else:
                proba=np.median(preds_proba[i1:i+delta+1],axis=0)
            preds.append(np.argmax(proba))
        for i,ind in enumerate(indices):
            if expressions[i]>=0:
                total_preds[hInd].append(preds[ind-1])

total_true=np.array(total_true)
for hInd,(isMean,delta) in enumerate(hyperparams):
    preds=np.array(total_preds[hInd])
    print('mean' if isMean else 'median',delta,'Acc:',(preds==total_true).mean(), 'F1:',f1_score(y_true=total_true,y_pred=preds, average="macro"))

mean 0 Acc: 0.500285172458044 F1: 0.3807067519117502
mean 2 Acc: 0.5134173641509703 F1: 0.3914238704567333
median 2 Acc: 0.5104373119644104 F1: 0.38878476888647095
mean 7 Acc: 0.5263855816805213 F1: 0.4018379088214337
median 7 Acc: 0.5236907019520055 F1: 0.39958196647074146


### Adaptive Frame Rate 

source code for the "Facial Expression Recognition with Adaptive Frame Rate based on Multiple Testing Correction" accepted at ICML 2023

In [20]:
deltas=[0,5,15,25,50,100,200,500]
total_true=[]
total_preds=[[] for _ in range(len(deltas))]
for videoname,(y_pred_expr,indices,expressions) in test_videos.items():
    for i,ind in enumerate(indices):
        if expressions[i]>=0:
            total_true.append(expressions[i])
    cur_ind=0
    preds_proba=[]
    for i in range(indices[-1]):
        if indices[cur_ind]-1==i:
            preds_proba.append(y_pred_expr[cur_ind])
            cur_ind+=1
        else:
            if cur_ind==0:
                preds_proba.append(y_pred_expr[cur_ind])
            else:
                w=(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                pred=w*y_pred_expr[cur_ind-1]+(1-w)*y_pred_expr[cur_ind]
                preds_proba.append(pred)
    
    preds_proba=np.array(preds_proba)
    for hInd,delta in enumerate(deltas):
        preds=[]
        for i in range(len(preds_proba)):
            i1=max(i-delta,0)
            proba=np.mean(preds_proba[i1:i+delta+1],axis=0)
            preds.append(np.argmax(proba))
        for i,ind in enumerate(indices):
            if expressions[i]>=0:
                total_preds[hInd].append(preds[ind-1])

total_true=np.array(total_true)
for hInd,delta in enumerate(deltas):
    preds=np.array(total_preds[hInd])
    print(delta,'Acc:',(preds==total_true).mean(), 'F1:',f1_score(y_true=total_true,y_pred=preds, average="macro"))

0 Acc: 0.500285172458044 F1: 0.3807067519117502
5 Acc: 0.5220830422197824 F1: 0.39793975387287783
15 Acc: 0.5355859581081659 F1: 0.40971928710644606
25 Acc: 0.5427259635264426 F1: 0.4173622193839083
50 Acc: 0.5499301327477792 F1: 0.4264244745156282
100 Acc: 0.5512134088089772 F1: 0.42621218115321136
200 Acc: 0.5428614204440135 F1: 0.4230475710116932
500 Acc: 0.5309911168779319 F1: 0.4218031114287871


#### Train 

In [21]:
delta=100

In [22]:
data_dir=os.path.join(DATA_DIR,'faces')
dirpath=os.path.join(DATA_DIR,'Third ABAW Annotations/EXPR_Classification_Challenge/Train_Set')
train_videos={}
for filename in tqdm(os.listdir(dirpath)):
    fn, ext = os.path.splitext(os.path.basename(filename))
    if ext.lower()=='.txt':
        X,indices,expressions=[],[],[]
        with open(os.path.join(dirpath,filename)) as f:
            lines = f.read().splitlines()
            prev_val=None
            for i,line in enumerate(lines):
                if i>0:
                    imagename=fn+'/'+get_names(i)+'.jpg'
                    if imagename in filename2featuresAll:
                        X.append(filename2featuresAll[imagename][0])
                        indices.append(i)
                        expressions.append(int(line))
        train_videos[fn]=(mlpModel.predict(np.array(X)),indices,np.array(expressions))
print(len(train_videos))

100%|██████████| 248/248 [00:56<00:00,  4.38it/s]

248


In [23]:
stride2scores={}
for stride in [200,100,50,25,10]:
    print('stride',stride)
    y_total_train,predictions,max_decision_values=[],[],[]
    for (y_pred_expr,indices,expressions) in train_videos.values():
        emotional_indices=[]
        for i,ind in enumerate(indices):
            if expressions[i]>=0:
                y_total_train.append(expressions[i])
                emotional_indices.append(ind-1)
        cur_ind=0
        preds_proba=[]
        for i in range(indices[-1]):
            if indices[cur_ind]-1==i:
                preds_proba.append(y_pred_expr[cur_ind])
                cur_ind+=1
            else:
                if cur_ind==0:
                    preds_proba.append(y_pred_expr[cur_ind])
                else:
                    w=(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                    pred=w*y_pred_expr[cur_ind-1]+(1-w)*y_pred_expr[cur_ind]
                    preds_proba.append(pred)

        preds_proba=np.array(preds_proba)
        for i in range(len(emotional_indices)):
            i1=max(emotional_indices[i]-delta,0)
            cur_preds=preds_proba[i1:emotional_indices[i]+delta+1:stride]
            proba=np.mean(cur_preds,axis=0)
            best_ind=np.argmax(proba)
            predictions.append(best_ind)
            max_decision_values.append(proba[best_ind])

    stride2scores[stride]=(np.array(y_total_train),np.array(predictions),np.array(max_decision_values))

stride 200
stride 100
stride 50
stride 25
stride 10


In [24]:
def get_threshold(stride,fpr_corrected):
    (y_total_train,predictions,max_decision_values)=stride2scores[stride]
    mistakes=max_decision_values[predictions!=y_total_train]
    best_threshold=-1
    for i,threshold in enumerate(sorted(max_decision_values[predictions==y_total_train])[::-1]):
        tpr=i/len(predictions)
        fpr=(mistakes>threshold).sum()/len(predictions)
        #print(threshold,fpr,tpr)
        if fpr>fpr_corrected:
            if best_threshold==-1:
                best_threshold=threshold
            print(stride,'best_threshold',best_threshold,i)
            break
        best_threshold=threshold
    return best_threshold

stride2threshold={}
for stride in stride2scores:
    fpr_corrected=0.05
    stride2threshold[stride]=get_threshold(stride,fpr_corrected)
stride2threshold[1]=0
print(stride2threshold)

200 best_threshold 0.68035233 412187
100 best_threshold 0.543008 494777
50 best_threshold 0.5162231 508738
25 best_threshold 0.4995789 518070
10 best_threshold 0.49043015 523199
{200: 0.68035233, 100: 0.543008, 50: 0.5162231, 25: 0.4995789, 10: 0.49043015, 1: 0}


#### Inference 

In [25]:
all_strides=[
    [200, 100, 50, 10, 1],
    [50, 25, 1],
    [50, 10, 1],
    [200,50,1],
    [100,50,1],
    [200,1],
    [100,1],
    [50,1]
]
for s in stride2threshold.keys():
    all_strides.append([s])

for strides in all_strides:
    print(strides)
    last_stride=strides[-1]

    total_true=[]
    total_preds=[]
    total_frames_processed,total_frames=0,0
    for (y_pred_expr,indices,expressions) in test_videos.values():
        emotional_indices=[]
        for i,ind in enumerate(indices):
            if expressions[i]>=0:
                total_true.append(expressions[i])
                emotional_indices.append(ind-1)
        cur_ind=0
        preds_proba=[]
        for i in range(indices[-1]):
            if indices[cur_ind]-1==i:
                preds_proba.append(y_pred_expr[cur_ind])
                cur_ind+=1
            else:
                if cur_ind==0:
                    preds_proba.append(y_pred_expr[cur_ind])
                else:
                    w=(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                    pred=w*y_pred_expr[cur_ind-1]+(1-w)*y_pred_expr[cur_ind]
                    preds_proba.append(pred)

        preds_proba=np.array(preds_proba)

        preds=-np.ones(len(emotional_indices))
        for stride in strides:
            threshold=stride2threshold[stride]
            for i in range(len(emotional_indices)):
                if preds[i]<0:
                    i1=max(emotional_indices[i]-delta,0)
                    cur_preds=preds_proba[i1:emotional_indices[i]+delta+1:stride]
                    proba=np.mean(cur_preds,axis=0)
                    best_ind=np.argmax(proba)
                    if proba[best_ind]>=threshold or stride==last_stride:
                        total_frames_processed+=len(cur_preds)
                        total_frames+=len(preds_proba[i1:emotional_indices[i]+delta+1])
                        preds[i]=best_ind
        for p in preds:
            total_preds.append(p)

    total_true=np.array(total_true)
    preds=np.array(total_preds)
    print('Acc:',(preds==total_true).mean(), 'F1:',f1_score(y_true=total_true,y_pred=preds, average="macro"))
    print(total_frames_processed,total_frames,total_frames_processed/total_frames)

[200, 100, 50, 10, 1]
Acc: 0.5402164458956554 F1: 0.41902184233325
11262799 55906719 0.2014569840880843
[50, 25, 1]
Acc: 0.549876662911896 F1: 0.42571232560361677
14860715 55906719 0.2658126834450793
[50, 10, 1]
Acc: 0.549947956026407 F1: 0.42578430647859183
13992904 55906719 0.25029020214904757
[200, 50, 1]
Acc: 0.5424015798554176 F1: 0.42027871842106457
16430120 55906719 0.29388453291276134
[100, 50, 1]
Acc: 0.5455919467297848 F1: 0.42246531152184047
15241987 55906719 0.2726324719574404
[200, 1]
Acc: 0.5432428386066473 F1: 0.42046923182790485
32631266 55906719 0.583673422151638
[100, 1]
Acc: 0.5461587269901473 F1: 0.42277716409351745
20393603 55906719 0.36477910642547273
[50, 1]
Acc: 0.5500085551737414 F1: 0.42583630334743344
18456371 55906719 0.3301279583228628
[200]
Acc: 0.4808471047866197 F1: 0.36236023497146447
551838 55906719 0.009870691928818074
[100]
Acc: 0.5232558139534884 F1: 0.40446631863369187
832370 55906719 0.014888550336856649
[50]
Acc: 0.537835255870988 F1: 0.416510939

In [26]:
#Complete example
all_strides=[
    [200, 100, 50, 10, 1],
    [50, 25, 1],
    [50, 10, 1],
    [200,50,1],
    [100,50,1],
    [200,1],
    [100,1],
    [50,1]
]
for s in stride2threshold.keys():
    all_strides.append([s])

#all_strides=[[200,1]]
for strides in all_strides:
    print(strides)
    last_stride=strides[-1]
    stride2threshold_cur={}
    for s_ind,stride in enumerate(strides):
        if stride!=last_stride:
            fpr_corrected=0.1*(s_ind+1)/(len(strides)) #0.08 #0.1 #
            print('stride',stride,fpr_corrected)
            stride2threshold_cur[stride]=get_threshold(stride,fpr_corrected)
        else:
            stride2threshold_cur[stride]=0

    total_true=[]
    total_preds=[]
    total_frames_processed,total_frames=0,0
    for (y_pred_expr,indices,expressions) in test_videos.values():
        emotional_indices=[]
        for i,ind in enumerate(indices):
            if expressions[i]>=0:
                total_true.append(expressions[i])
                emotional_indices.append(ind-1)
        cur_ind=0
        preds_proba=[]
        for i in range(indices[-1]):
            if indices[cur_ind]-1==i:
                preds_proba.append(y_pred_expr[cur_ind])
                cur_ind+=1
            else:
                if cur_ind==0:
                    preds_proba.append(y_pred_expr[cur_ind])
                else:
                    w=(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                    pred=w*y_pred_expr[cur_ind-1]+(1-w)*y_pred_expr[cur_ind]
                    preds_proba.append(pred)

        preds_proba=np.array(preds_proba)

        preds=-np.ones(len(emotional_indices))
        for stride in strides:
            threshold=stride2threshold_cur[stride]
            for i in range(len(emotional_indices)):
                if preds[i]<0:
                    i1=max(emotional_indices[i]-delta,0)
                    cur_preds=preds_proba[i1:emotional_indices[i]+delta+1:stride]
                    proba=np.mean(cur_preds,axis=0)
                    best_ind=np.argmax(proba)
                    if proba[best_ind]>=threshold or stride==last_stride:
                        total_frames_processed+=len(cur_preds)
                        total_frames+=len(preds_proba[i1:emotional_indices[i]+delta+1])
                        preds[i]=best_ind
        for p in preds:
            total_preds.append(p)

    total_true=np.array(total_true)
    preds=np.array(total_preds)
    print('Acc:',(preds==total_true).mean(), 'F1:',f1_score(y_true=total_true,y_pred=preds, average="macro"))
    print(total_frames_processed,total_frames,total_frames_processed/total_frames)

[200, 100, 50, 10, 1]
stride 200 0.02
200 best_threshold 0.8820417 341051
stride 100 0.04
100 best_threshold 0.58247334 482013
stride 50 0.06000000000000001
50 best_threshold 0.4818677 517646
stride 10 0.08
Acc: 0.5439914163090128 F1: 0.42174291492447946
2069089 55906719 0.03700966604747454
[50, 25, 1]
stride 50 0.03333333333333333
50 best_threshold 0.5796947 487628
stride 25 0.06666666666666667
25 best_threshold 0.41966972 532157
Acc: 0.5496378309782841 F1: 0.4252566581975647
9573269 55906719 0.17123646622868352
[50, 10, 1]
stride 50 0.03333333333333333
50 best_threshold 0.5796947 487628
stride 10 0.06666666666666667
10 best_threshold 0.39557225 537192
Acc: 0.5503400681562175 F1: 0.4257837197103246
8673566 55906719 0.15514353471538905
[200, 50, 1]
stride 200 0.03333333333333333
200 best_threshold 0.78248155 382474
stride 50 0.06666666666666667
50 best_threshold 0.44965786 523290
Acc: 0.5435458343433192 F1: 0.4207483426245095
11408607 55906719 0.20406504270086034
[100, 50, 1]
stride 10

### Save test results 

In [20]:
with open(os.path.join(DATA_DIR,'Expression_Classification_Challenge_test_set_release.txt'),'r') as f:
    test_set_videos=f.read().splitlines()
print(len(test_set_videos),test_set_videos[:5])

228 ['14-30-1920x1080', '16-30-1920x1080', '40-30-1280x720', '43-30-406x720', '44-25-426x240']


In [21]:
data_dir=os.path.join(DATA_DIR,'faces')
test_videos={}
for videoname in tqdm(os.listdir(data_dir)):
    if videoname in test_set_videos:
        X,indices,filenames=[],[],[]
        images=[img_name for img_name in os.listdir(os.path.join(data_dir,videoname)) if img_name.lower().endswith('.jpg')]
        for img_name in sorted(images, key=compare_filenames):
            k=videoname+'/'+img_name
            if k in filename2featuresAll:
                X.append(filename2featuresAll[k][0])
                indices.append(int(img_name[:-4]))
                filenames.append(k)
        test_videos[videoname]=(mlpModel.predict(np.array(X)),indices,filenames)
print(len(test_videos))

100%|██████████| 564/564 [00:55<00:00, 10.23it/s]

228


In [29]:
delta=0 # 0 2 4 7
resdir=os.path.join(DATA_DIR,'test_results/EXPR/1_single') #2_mean5 5_med15
if not os.path.exists(resdir):
    os.makedirs(resdir)
header = 'Neutral,Anger,Disgust,Fear,Happiness,Sadness,Surprise,Other\n'
for videoname,(y_pred_expr,indices,filenames) in tqdm(test_videos.items()):
    cur_ind=0
    preds_proba=[]
    for i in range(indices[-1]):
        if indices[cur_ind]-1==i:
            preds_proba.append(y_pred_expr[cur_ind])
            cur_ind+=1
        else:
            if cur_ind==0:
                preds_proba.append(y_pred_expr[cur_ind])
            else:
                w=(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                pred=w*y_pred_expr[cur_ind-1]+(1-w)*y_pred_expr[cur_ind]
                preds_proba.append(pred)
    
    pred=y_pred_expr[cur_ind-1]
    vn=videoname
    if videoname.endswith('_left'):
        vn=videoname[:-5]
    elif videoname.endswith('_right'):
        vn=videoname[:-6]
    for _ in range(indices[-1],video2len[vn]):
        preds_proba.append(pred)

    preds_proba=np.array(preds_proba)
    with open(os.path.join(resdir,videoname+'.txt'), 'w') as f:
        f.write(header)
        for i in range(len(preds_proba)):
            i1=max(i-delta,0)
            proba=np.mean(preds_proba[i1:i+delta+1],axis=0)
            #proba=np.median(preds_proba[i1:i+delta+1],axis=0)
            f.write(str(np.argmax(proba))+'\n')

100%|██████████| 228/228 [00:08<00:00, 27.56it/s]


## Valence-Arousal 

In [86]:
def get_image2VA(dirname):
    dirpath=os.path.join(DATA_DIR,'Third ABAW Annotations/VA_Estimation_Challenge/',dirname)
    num_missed=0
    X,y=[],[]
    for filename in os.listdir(dirpath):
        fn, ext = os.path.splitext(os.path.basename(filename))
        if ext.lower()=='.txt':
            with open(os.path.join(dirpath,filename)) as f:
                lines = f.read().splitlines()
                for i,line in enumerate(lines):
                    if i>0:
                        splitted_line=line.split(',')
                        valence=float(splitted_line[0])
                        arousal=float(splitted_line[1])
                        if valence>=-1 and arousal>=-1:
                            imagename=fn+'/'+get_names(i)+'.jpg'
                            if imagename in filename2featuresAll:
                                X.append(filename2featuresAll[imagename][1])
                                y.append((valence,arousal))
                            else:
                                num_missed+=1
    X=np.array(X)
    y=np.array(y)
    print(X.shape,y.shape,num_missed)
    return X,y

X_train,y_train=get_image2VA('Train_Set')
X_val,y_val=get_image2VA('Validation_Set')

(1555919, 8) (1555919, 2) 25265
(338755, 8) (338755, 2) 5959


In [87]:
def print_ccc():
    y_val_preds=mlpModel.predict(X_val)
    gt_V=y_val[:,0]
    gt_A=y_val[:,1]
    pred_V=y_val_preds[:,0]
    pred_A=y_val_preds[:,1]
    print(metric_for_VA(gt_V,gt_A,pred_V,pred_A))
    print(CCC_numpy(gt_V,pred_V),CCC_numpy(gt_A,pred_A))

In [88]:
batch_size=512 #128
mlpModel=Sequential()
if True:
    mlpModel.add(Dense(2, input_shape=X_train.shape[1:],activation='tanh',use_bias=True,kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size)))
else:
    mlpModel.add(Dense(128, input_shape=X_train.shape[1:],activation='relu')) #256
    mlpModel.add(Dense(2,activation='tanh'))

In [27]:
mlpModel.compile(optimizer=Adam(lr=1e-3), loss=CCC_VA, metrics=['mae','mse'])
mlpModel.summary()

save_best_model = SaveBestModel('val_loss',False)
mlpModel.fit(X_train,y_train, batch_size=batch_size, epochs=20, verbose=1, callbacks=[save_best_model], validation_data=(X_val,y_val))
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 2)                 18        
Total params: 18
Trainable params: 18
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
3039/3039 [==============================] - 12s 4ms/step - loss: 0.6652 - mae: 0.4770 - mse: 0.3555 - val_loss: 0.8230 - val_mae: 0.2781 - val_mse: 0.1300
Epoch 2/20
3039/3039 [==============================] - 11s 4ms/step - loss: 0.5047 - mae: 0.2775 - mse: 0.1276 - val_loss: 0.8233 - val_mae: 0.2788 - val_mse: 0.1306
Epoch 3/20
3039/3039 [==============================] - 10s 3ms/step - loss: 0.5056 - mae: 0.2774 - mse: 0.1276 - val_loss: 0.8243 - val_mae: 0.2821 - val_mse: 0.1330
Epoch 4/20
3039/3039 [==============================] - 11s 4ms/step - loss: 0.5057 - mae: 0.2778 - mse: 0.1279 - val_loss: 0.8234 - val_mae: 0.2775 - 

In [28]:
print_ccc()
print('Best weights:')
mlpModel.set_weights(best_model_weights)
print_ccc()

(0.4298415239084466, 0.4922025796049645, 0.4610220517567055)
0.42984280225697313 0.4922040352852844
Best weights:
(0.42974953297503016, 0.49796161300946734, 0.4638555729922488)
0.42975080214878003 0.4979630952849663


In [89]:
if False:
    mlpModel.save_weights('va_enet0_vgaf.h5') #(0.42974953297503016, 0.49796161300946734, 0.4638555729922488)
else:
    mlpModel.load_weights('va_enet0_vgaf.h5')
    print_ccc()

(0.42974953297503016, 0.49796161300946734, 0.4638555729922488)
0.42975080214878003 0.4979630952849663


### Smooth validation predictions

In [92]:
data_dir=os.path.join(DATA_DIR,'faces')
dirpath=os.path.join(DATA_DIR,'Third ABAW Annotations/VA_Estimation_Challenge/Validation_Set')
test_videos={}
for filename in tqdm(os.listdir(dirpath)):
    fn, ext = os.path.splitext(os.path.basename(filename))
    if ext.lower()=='.txt':
        X,indices,y_true=[],[],[]
        with open(os.path.join(dirpath,filename)) as f:
            lines = f.read().splitlines()
            prev_val=None
            for i,line in enumerate(lines):
                if i>0:
                    imagename=fn+'/'+get_names(i)+'.jpg'
                    if imagename in filename2featuresAll:
                        X.append(filename2featuresAll[imagename][1])
                        indices.append(i)
                        
                        splitted_line=line.split(',')
                        valence=float(splitted_line[0])
                        arousal=float(splitted_line[1])
                        y_true.append((valence,arousal))
                        
        test_videos[fn]=(mlpModel.predict(np.array(X)),indices,np.array(y_true))
print(len(test_videos))

100%|██████████| 71/71 [00:10<00:00,  6.77it/s]

71


In [97]:
hyperparams=[(isMean,delta) for delta in [0,2,7]  for isMean in [1,0] if not (isMean==0 and delta==0)]
total_true=[]
total_preds=[[] for _ in range(len(hyperparams))]
for videoname,(y_pred_va,indices,y_true) in test_videos.items():
    for i,ind in enumerate(indices):
        if y_true[i][0]>=-1 and y_true[i][1]>=-1:
            total_true.append(y_true[i])
    cur_ind=0
    preds=[]
    for i in range(indices[-1]):
        if indices[cur_ind]-1==i:
            preds.append(y_pred_va[cur_ind])
            cur_ind+=1
        else:
            if cur_ind==0:
                preds.append(y_pred_va[cur_ind])
            else:
                w=(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                pred=w*y_pred_va[cur_ind-1]+(1-w)*y_pred_va[cur_ind]
                preds.append(pred)
    
    preds=np.array(preds)
    for hInd,(isMean,delta) in enumerate(hyperparams):
        cur_preds=[]
        for i in range(len(preds)):
            i1=max(i-delta,0)
            if isMean:
                pred=np.mean(preds[i1:i+delta+1],axis=0)
            else:
                pred=np.median(preds[i1:i+delta+1],axis=0)
            cur_preds.append(pred)
        for i,ind in enumerate(indices):
            if y_true[i][0]>=-1 and y_true[i][1]>=-1:
                total_preds[hInd].append(cur_preds[ind-1])

total_true=np.array(total_true)
gt_V=total_true[:,0]
gt_A=total_true[:,1]

for hInd,(isMean,delta) in enumerate(hyperparams):
    preds=np.array(total_preds[hInd])
    pred_V=preds[:,0]
    pred_A=preds[:,1]
    print('mean' if isMean else 'median',delta,metric_for_VA(gt_V,gt_A,pred_V,pred_A))

mean 0 (0.4297495329925397, 0.4979616130077603, 0.46385557300015)
mean 2 (0.43746115316549106, 0.5131634836328756, 0.4753123183991833)
median 2 (0.43544421989237614, 0.5072473979359517, 0.47134580891416394)
mean 7 (0.44849536890896224, 0.5353431416480636, 0.49191925527851293)
median 7 (0.44590019769035444, 0.527161073024317, 0.48653063535733576)


### Save test results 

In [34]:
with open(os.path.join(DATA_DIR,'Valence_Arousal_Estimation_Challenge_test_set_release.txt'),'r') as f:
    test_set_videos=f.read().splitlines()
print(len(test_set_videos),test_set_videos[:5])

152 ['2-30-640x360', '3-25-1920x1080', '6-30-1920x1080_left', '6-30-1920x1080_right', '11-24-1920x1080']


In [35]:
data_dir=os.path.join(DATA_DIR,'faces')
test_videos={}
for videoname in tqdm(os.listdir(data_dir)):
    if videoname in test_set_videos:
        X,indices=[],[]
        images=[img_name for img_name in os.listdir(os.path.join(data_dir,videoname)) if img_name.lower().endswith('.jpg')]
        for img_name in sorted(images, key=compare_filenames):
            k=videoname+'/'+img_name
            if k in filename2featuresAll:
                X.append(filename2featuresAll[k][1])
                indices.append(int(img_name[:-4]))
        test_videos[videoname]=(mlpModel.predict(np.array(X)),indices)
print(len(test_videos))

100%|██████████| 564/564 [00:24<00:00, 23.28it/s]

152


In [41]:
delta=2 #0 2 4 7
resdir=os.path.join(DATA_DIR,'test_results/VA/2_mean5') #2_mean5 5_med15
if not os.path.exists(resdir):
    os.makedirs(resdir)
header = 'valence,arousal\n'

for videoname,(y_pred_va,indices) in test_videos.items():
    cur_ind=0
    preds=[]
    for i in range(indices[-1]):
        if indices[cur_ind]-1==i:
            preds.append(y_pred_va[cur_ind])
            cur_ind+=1
        else:
            if cur_ind==0:
                preds.append(y_pred_va[cur_ind])
            else:
                w=(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                pred=w*y_pred_va[cur_ind-1]+(1-w)*y_pred_va[cur_ind]
                preds.append(pred)

    pred=y_pred_va[cur_ind-1]
    vn=videoname
    if videoname.endswith('_left'):
        vn=videoname[:-5]
    elif videoname.endswith('_right'):
        vn=videoname[:-6]
    for _ in range(indices[-1],video2len[vn]):
        preds.append(pred)
    preds=np.array(preds)
    with open(os.path.join(resdir,videoname+'.txt'), 'w') as f:
        f.write(header)
        for i in range(len(preds)):
            i1=max(i-delta,0)
            pred=np.mean(preds[i1:i+delta+1],axis=0)
            #pred=np.median(preds[i1:i+delta+1],axis=0)
            f.write(str(pred[0])+','+str(pred[1])+'\n')


## AU 

In [98]:
def get_image2AU(dirname):
    dirpath=os.path.join(DATA_DIR,'Third ABAW Annotations/AU_Detection_Challenge/',dirname)
    num_missed=0
    X,y=[],[]
    for filename in os.listdir(dirpath):
        fn, ext = os.path.splitext(os.path.basename(filename))
        if ext.lower()=='.txt':
            with open(os.path.join(dirpath,filename)) as f:
                lines = f.read().splitlines()
                for i,line in enumerate(lines):
                    if i>0:
                        splitted_line=line.split(',')
                        aus=list(map(int,splitted_line))
                        if min(aus)>=0:
                            imagename=fn+'/'+get_names(i)+'.jpg'
                            if imagename in filename2featuresAll:
                                X.append(filename2featuresAll[imagename][0])
                                y.append(aus)
                            else:
                                num_missed+=1
    X=np.array(X)
    y=np.array(y)
    print(X.shape,y.shape,num_missed)
    return X,y

X_train,y_train=get_image2AU('Train_Set')
X_val,y_val=get_image2AU('Validation_Set')

(1356861, 1280) (1356861, 12) 2829
(445836, 1280) (445836, 12) 9


In [99]:
def print_au():
    y_val_preds=mlpModel.predict(X_val)
    new_pred = ((y_val_preds >= 0.5) * 1)
    print(np.mean([f1_score(y_true=y_val[:,i],y_pred=new_pred[:,i]) for i in range(y_val_preds.shape[1])]))
    print(f1_score_max(y_val,y_val_preds,thresh=np.arange(0.1,1,0.1)))


In [49]:
num_labels=y_train.shape[1]
print(num_labels)
from sklearn.utils.class_weight import compute_class_weight
class_weights = np.empty([num_labels, 2])
for i in range(num_labels):
    neg, pos = np.bincount(y_train[:, i])
    total = neg + pos
    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)

    class_weights[i][0]=weight_for_0
    class_weights[i][1]=weight_for_1
    #class_weights[i] = compute_class_weight('balanced', [0,1], y_train[:, i])
print(class_weights)

12
[[ 0.56699737  4.23148962]
 [ 0.52623698 10.02853659]
 [ 0.59099307  3.24746184]
 [ 0.68134051  1.87862194]
 [ 0.82949071  1.25874673]
 [ 0.7628283   1.45119134]
 [ 0.66223812  2.04094492]
 [ 0.51446878 17.77857704]
 [ 0.51624061 15.8935131 ]
 [ 0.51314264 19.52205628]
 [ 1.3391547   0.79791885]
 [ 0.54019984  6.71892981]]


In [50]:
if False:
    loss='binary_crossentropy'
    #loss='hinge'
else:
    import tensorflow.keras.backend as K
    def get_weighted_loss(weights):
        def weighted_loss(y_true, y_pred):
            y_true=tf.cast(y_true, tf.float32)
            ce=K.binary_crossentropy(y_true, y_pred)
            return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*ce, axis=-1)
        return weighted_loss
    loss=get_weighted_loss(class_weights)

In [51]:
metrics=[tf.keras.metrics.AUC(multi_label=True,name='auc'), tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()] # 

In [100]:
batch_size=512 #128
mlpModel=Sequential()
if False:
    mlpModel.add(Dense(y_train.shape[1], input_shape=X_train.shape[1:],activation='sigmoid',use_bias=True,kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size)))
else:
    mlpModel.add(Dense(128, input_shape=X_train.shape[1:],activation='relu')) #256
    mlpModel.add(Dense(y_train.shape[1],activation='sigmoid'))

In [53]:
mlpModel.compile(optimizer=Adam(lr=1e-3), loss=loss, metrics=metrics)
mlpModel.summary()

save_best_model = SaveBestModel('val_loss',False)
mlpModel.fit(X_train,y_train, batch_size=batch_size, epochs=20, verbose=1, callbacks=[save_best_model], validation_data=(X_val,y_val))
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               163968    
_________________________________________________________________
dense_10 (Dense)             (None, 12)                1548      
Total params: 165,516
Trainable params: 165,516
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
2651/2651 [==============================] - 18s 6ms/step - loss: 0.3631 - auc: 0.9146 - binary_accuracy: 0.8374 - recall: 0.8133 - precision: 0.5637 - val_loss: 0.6988 - val_auc: 0.8226 - val_binary_accuracy: 0.8227 - val_recall: 0.7469 - val_precision: 0.5523
Epoch 2/20
2651/2651 [==============================] - 16s 6ms/step - loss: 0.2615 - auc: 0.9556 - binary_accuracy: 0.8854 - recall: 0.8644 - precision: 0.6591 - val_loss: 0.7778 - val_auc: 0.8188 - val_binary_accuracy: 0.8338 - val_rec

In [54]:
print_au()
print('Best weights:')
mlpModel.set_weights(best_model_weights)
print_au()

0.4864278055496279
(0.49427122676733243, 0.6416666666666666, array([0.51627374, 0.40038851, 0.49818827, 0.58588789, 0.7150855 ,
       0.72659929, 0.69216924, 0.28844102, 0.17527699, 0.18035356,
       0.82694271, 0.32564799]), array([0.8, 0.7, 0.7, 0.6, 0.5, 0.6, 0.6, 0.8, 0.9, 0.6, 0.3, 0.6]), array([0.86745799, 0.88922608, 0.84654896, 0.79186741, 0.74430957,
       0.79999821, 0.84309701, 0.9461506 , 0.95742605, 0.94134615,
       0.7577607 , 0.82674122]))
Best weights:
0.5018743464268921
(0.5326505266567211, 0.6666666666666666, array([0.5586506 , 0.46318824, 0.57471749, 0.61564439, 0.73792177,
       0.74323626, 0.71372861, 0.34674426, 0.18797831, 0.2369548 ,
       0.83616326, 0.37687833]), array([0.8, 0.8, 0.8, 0.6, 0.5, 0.5, 0.6, 0.8, 0.7, 0.9, 0.3, 0.7]), array([0.88197678, 0.90478562, 0.864181  , 0.79781579, 0.77860469,
       0.80720489, 0.85493993, 0.95065226, 0.95263954, 0.95683615,
       0.77205968, 0.84885698]))


In [101]:
if False:
    mlpModel.save_weights('au_enet0_vgaf.h5') #0.507568027779308
else:
    mlpModel.load_weights('au_enet0_vgaf.h5')
    print_au()

0.507568027779308
(0.5367145284583944, 0.65, array([0.56438654, 0.47163507, 0.57207889, 0.61677155, 0.74818918,
       0.74534595, 0.71942028, 0.35743029, 0.19360184, 0.24140214,
       0.83680711, 0.37350551]), array([0.8, 0.8, 0.7, 0.5, 0.5, 0.5, 0.6, 0.8, 0.8, 0.8, 0.3, 0.7]), array([0.8884859 , 0.90530374, 0.8692748 , 0.79757579, 0.78782557,
       0.80682583, 0.8569631 , 0.94949937, 0.94323473, 0.95116814,
       0.77594452, 0.85026333]))


In [102]:
pred=mlpModel.predict(X_val)
#thresholds=0.5
#thresholds=np.array([0.7, 0.8, 0.7, 0.6, 0.5, 0.5, 0.6, 0.8, 0.7, 0.8, 0.3, 0.7]) #au_enet0_7 - 0.5228095834398018
thresholds=np.array([0.8, 0.8, 0.7, 0.5, 0.5, 0.5, 0.6, 0.8, 0.8, 0.8, 0.3, 0.7]) #vgaf 0.5367145284583944
new_pred = ((pred >= thresholds) * 1)
print(np.mean([f1_score(y_true=y_val[:,i],y_pred=new_pred[:,i]) for i in range(pred.shape[1])]))

0.5367145284583944


mobilenet
features
logreg: 0.47297223875271305
(0.5239126048120428, 0.6166666666666667
dense: 0.4776362901110953

emotions
logreg: 0.43245972823656503
0.44210386971177873, 0.5083333333333334
dense: 0.4515366088060122
(0.4866808135583551, 0.6083333333333333


enetb0_7:
features
logreg: 0.47423963532530294
(0.5172051728975023, 0.6
dense: 0.4909581965862712
(0.5183018786548269, 0.6749999999999999

enet_b0_8_best_afew
features
dense: 0.4850984023504916
(0.5154674855367459, 0.6749999999999999

enet_b0_8_best_vgaf
features
dense: 0.507568027779308
(0.5367145284583944, 0.65


enetb2_8:
features
logreg: 0.467894360096913
(0.5026441356600955, 0.5916666666666666
dense: 0.48209903459210973
(0.5121049982984248, 0.6666666666666666

### Smooth validation predictions

In [104]:
data_dir=os.path.join(DATA_DIR,'faces')
dirpath=os.path.join(DATA_DIR,'Third ABAW Annotations/AU_Detection_Challenge/Validation_Set')
test_videos={}
for filename in tqdm(os.listdir(dirpath)):
    fn, ext = os.path.splitext(os.path.basename(filename))
    if ext.lower()=='.txt':
        X,indices,y_true=[],[],[]
        with open(os.path.join(dirpath,filename)) as f:
            lines = f.read().splitlines()
            prev_val=None
            for i,line in enumerate(lines):
                if i>0:
                    imagename=fn+'/'+get_names(i)+'.jpg'
                    if imagename in filename2featuresAll:
                        X.append(filename2featuresAll[imagename][0])
                        indices.append(i)
                        splitted_line=line.split(',')
                        aus=list(map(int,splitted_line))
                        y_true.append(aus)
        test_videos[fn]=(mlpModel.predict(np.array(X)),indices,np.array(y_true))
print(len(test_videos))

100%|██████████| 105/105 [00:23<00:00,  4.47it/s]

105


In [105]:
hyperparams=[(isMean,delta) for delta in [0,2,7]  for isMean in [1,0] if not (isMean==0 and delta==0)]
total_true=[]
total_preds=[[] for _ in range(len(hyperparams))]
for videoname,(y_pred_aus,indices,y_true) in test_videos.items():
    for i,ind in enumerate(indices):
        if min(y_true[i])>=0:
            total_true.append(y_true[i])
    cur_ind=0
    preds=[]
    for i in range(indices[-1]):
        if indices[cur_ind]-1==i:
            preds.append(y_pred_aus[cur_ind])
            cur_ind+=1
        else:
            if cur_ind==0:
                preds.append(y_pred_aus[cur_ind])
            else:
                w=(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                pred=w*y_pred_aus[cur_ind-1]+(1-w)*y_pred_aus[cur_ind]
                preds.append(pred)
    
    preds=np.array(preds)
    for hInd,(isMean,delta) in enumerate(hyperparams):
        cur_preds=[]
        for i in range(len(preds)):
            i1=max(i-delta,0)
            if isMean:
                pred=np.mean(preds[i1:i+delta+1],axis=0)
            else:
                pred=np.median(preds[i1:i+delta+1],axis=0)
            aus=(pred>=thresholds)*1
            cur_preds.append(aus)
        for i,ind in enumerate(indices):
            if min(y_true[i])>=0:
                total_preds[hInd].append(cur_preds[ind-1])

total_true=np.array(total_true)
for hInd,(isMean,delta) in enumerate(hyperparams):
    preds=np.array(total_preds[hInd])
    print('mean' if isMean else 'median',delta,np.mean([f1_score(y_true=total_true[:,i],y_pred=preds[:,i]) for i in range(preds.shape[1])]))


mean 0 0.5367145284583944
mean 2 0.5446927222162641
median 2 0.5429543082968176
mean 7 0.5445119128957571
median 7 0.5478115868229798


### Save test results

In [47]:
with open(os.path.join(DATA_DIR,'Action_Unit_Detection_Challenge_test_set_release.txt'),'r') as f:
    test_set_videos=f.read().splitlines()
print(len(test_set_videos),test_set_videos[:5])

141 ['2-30-640x360', '3-25-1920x1080', '6-30-1920x1080_left', '6-30-1920x1080_right', '11-24-1920x1080']


In [48]:
data_dir=os.path.join(DATA_DIR,'faces')
test_videos={}
for videoname in tqdm(os.listdir(data_dir)):
    if videoname in test_set_videos:
        X,indices=[],[]
        images=[img_name for img_name in os.listdir(os.path.join(data_dir,videoname)) if img_name.lower().endswith('.jpg')]
        for img_name in sorted(images, key=compare_filenames):
            k=videoname+'/'+img_name
            if k in filename2featuresAll:
                X.append(filename2featuresAll[k][0])
                indices.append(int(img_name[:-4]))
        test_videos[videoname]=(mlpModel.predict(np.array(X)),indices)
print(len(test_videos))

100%|██████████| 564/564 [00:36<00:00, 15.61it/s]

141


In [53]:
delta=0 #0 2 4 7
resdir=os.path.join(DATA_DIR,'test_results/AU/1_single') #2_mean5 5_med15
if not os.path.exists(resdir):
    os.makedirs(resdir)
header = 'AU1,AU2,AU4,AU6,AU7,AU10,AU12,AU15,AU23,AU24,AU25,AU26\n'

for videoname,(y_pred_aus,indices) in tqdm(test_videos.items()):
    cur_ind=0
    preds=[]
    for i in range(indices[-1]):
        if indices[cur_ind]-1==i:
            preds.append(y_pred_aus[cur_ind])
            cur_ind+=1
        else:
            if cur_ind==0:
                preds.append(y_pred_aus[cur_ind])
            
            else:
                w=(i-indices[cur_ind-1]+1)/(indices[cur_ind]-indices[cur_ind-1])
                pred=w*y_pred_aus[cur_ind-1]+(1-w)*y_pred_aus[cur_ind]
                preds.append(pred)
                
    pred=y_pred_aus[cur_ind-1]
    vn=videoname
    if videoname.endswith('_left'):
        vn=videoname[:-5]
    elif videoname.endswith('_right'):
        vn=videoname[:-6]
    for _ in range(indices[-1],video2len[vn]):
        preds.append(pred)
    
    preds=np.array(preds)
    with open(os.path.join(resdir,videoname+'.txt'), 'w') as f:
        f.write(header)
        for i in range(len(preds)):
            i1=max(i-delta,0)
            pred=np.mean(preds[i1:i+delta+1],axis=0)
            #pred=np.median(preds[i1:i+delta+1],axis=0)
            aus=(pred>=thresholds)*1
            f.write(','.join(map(str,aus))+'\n')

100%|██████████| 141/141 [00:10<00:00, 13.28it/s]


## Multi-task challenge 

In [14]:
def get_image2all(filename):
    with open(os.path.join(DATA_DIR,'Third ABAW Annotations/MTL_Challenge/'+filename)) as f:
        mtl_lines = f.read().splitlines()
    num_missed=0
    X,y_va,y_expr,y_aus=[],[],[],[]
    masks_va,masks_expr,masks_aus=[],[],[]
    for line in mtl_lines[1:]:
        splitted_line=line.split(',')
        imagename=splitted_line[0]
        valence=float(splitted_line[1])
        arousal=float(splitted_line[2])
        expression=int(splitted_line[3])
        aus=list(map(int,splitted_line[4:]))
        
        mask_VA=(valence>-5 and arousal>-5)
        if not mask_VA:
            valence=arousal=0
            
        mask_expr=(expression>-1)
        if not mask_expr:
            expression=0
            
        mask_aus=min(aus)>=0
        if not mask_aus:
            aus=[0]*len(aus)
        if mask_VA or mask_expr or mask_aus:
            if imagename in filename2featuresAll:
                #X.append(filename2featuresAll[imagename][0])
                X.append(np.concatenate((filename2featuresAll[imagename][0],filename2featuresAll[imagename][1])))
                y_va.append((valence,arousal))
                masks_va.append(mask_VA)
                
                y_expr.append(expression)
                masks_expr.append(mask_expr)
                
                y_aus.append(aus)
                masks_aus.append(mask_aus)
            else:
                num_missed+=1
    X=np.array(X)
    y_va=np.array(y_va)
    y_expr=np.array(y_expr)
    y_aus=np.array(y_aus)
    masks_va=np.array(masks_va).astype(np.float32)
    masks_expr=np.array(masks_expr).astype(np.float32)
    masks_aus=np.array(masks_aus).astype(np.float32)
    print(X.shape,y_va.shape,y_expr.shape,y_aus.shape,masks_va.shape,num_missed)
    return X,y_va,y_expr,y_aus,masks_va,masks_expr,masks_aus

X_train,y_va_train,y_expr_train,y_aus_train,masks_va_train,masks_expr_train,masks_aus_train=get_image2all('train_set.txt')
X_val,y_va_val,y_expr_val,y_aus_val,masks_va_val,masks_expr_val,masks_aus_val=get_image2all('validation_set.txt')
TRAIN_VAL=False

(142225, 1416) (142225, 2) (142225,) (142225, 12) (142225,) 2666
(26876, 1416) (26876, 2) (26876,) (26876, 12) (26876,) 211


In [35]:
if False:
    TRAIN_VAL=True
    X_train=np.concatenate((X_train,X_val))
    y_va_train=np.concatenate((y_va_train,y_va_val))
    y_expr_train=np.concatenate((y_expr_train,y_expr_val))
    y_aus_train=np.concatenate((y_aus_train,y_aus_val))
    masks_va_train=np.concatenate((masks_va_train,masks_va_val))
    masks_expr_train=np.concatenate((masks_expr_train,masks_expr_val))
    masks_aus_train=np.concatenate((masks_aus_train,masks_aus_val))
    print(X_train.shape,y_va_train.shape,y_expr_train.shape,y_aus_train.shape,
          masks_va_train.shape,masks_expr_train.shape,masks_aus_train.shape)

(169101, 1416) (169101, 2) (169101,) (169101, 12) (169101,) (169101,) (169101,)


In [36]:
(unique, counts) = np.unique(y_expr_train[masks_expr_train==1.].astype(int), return_counts=True)
num_classes=len(unique)
emo_cw=1/counts
emo_cw/=emo_cw.min()
emo_class_weights = {i:cwi for i,cwi in zip(unique,emo_cw)}
print(counts, emo_class_weights, num_classes, unique)
print(emo_cw)

[25844  5042  3857  4376 21892  9375  6261 29453] {0: 1.1396455657019038, 1: 5.841531138437128, 2: 7.636245786880996, 3: 6.730575868372943, 4: 1.345377306778732, 5: 3.1416533333333336, 6: 4.7042006069318, 7: 1.0} 8 [0 1 2 3 4 5 6 7]
[1.13964557 5.84153114 7.63624579 6.73057587 1.34537731 3.14165333
 4.70420061 1.        ]


In [37]:
num_labels=y_aus_train.shape[1]
print(num_labels)
from sklearn.utils.class_weight import compute_class_weight
aus_class_weights = np.empty([num_labels, 2])
for i in range(num_labels):
    neg, pos = np.bincount(y_aus_train[masks_aus_train==1., i])
    total = neg + pos
    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)

    aus_class_weights[i][0]=weight_for_0
    aus_class_weights[i][1]=weight_for_1
print(aus_class_weights)

12
[[ 0.61478236  2.67803499]
 [ 0.54767511  5.74382689]
 [ 0.62284785  2.53503769]
 [ 0.71610871  1.6568252 ]
 [ 0.8608845   1.19274241]
 [ 0.80939227  1.30803571]
 [ 0.68422579  1.85703045]
 [ 0.51277199 20.0740798 ]
 [ 0.51470367 17.50256203]
 [ 0.52343372 11.16838754]
 [ 1.65902758  0.71569806]
 [ 0.55872706  4.7569816 ]]


In [38]:
def loss_va(y_true, y_pred):
    res=1-0.5*(CCC(y_true[:,0], y_pred[:,0])+CCC(y_true[:,1], y_pred[:,1]))
    return res

In [39]:
class WeightedExprSCCE(tf.keras.losses.Loss):
    def __init__(self, class_weight, from_logits=False, name='expr_scce'):
        if class_weight is None or all(v == 1. for v in class_weight):
            self.class_weight = None
        else:
            self.class_weight = tf.convert_to_tensor(class_weight,
                dtype=tf.float32)
        self.reduction = tf.keras.losses.Reduction.NONE
        self.unreduced_scce = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=from_logits, name=name,
            reduction=self.reduction)

    def __call__(self, y_true, y_pred, sample_weight=None):
        loss = self.unreduced_scce(y_true, y_pred, sample_weight)
        if self.class_weight is not None:
            weight_mask = tf.gather(self.class_weight, y_true)
            loss = tf.math.multiply(loss, weight_mask)
        loss=K.mean(loss)
        return loss
loss_expr=WeightedExprSCCE(emo_cw)

In [40]:
def get_weighted_loss_aus(weights):
    def weighted_loss(y_true, y_pred):
        y_true=tf.cast(y_true, tf.float32)
        ce=K.binary_crossentropy(y_true, y_pred)
        res=K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*ce)
        return res
    return weighted_loss
loss_aus=get_weighted_loss_aus(aus_class_weights)

In [41]:
metrics=[tf.keras.metrics.AUC(multi_label=True,name='auc'), tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()] # 

In [42]:
batch_size=256 #128
img = tf.keras.Input(shape=X_train.shape[1:])
mask1 = tf.keras.Input(shape=(1,))
mask2 = tf.keras.Input(shape=(1,))
mask3 = tf.keras.Input(shape=(1,))
x=img
#x=Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
va_out = Dense(2, activation='tanh',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
va_out_masked=tf.keras.layers.Multiply(name='va_out')([va_out,mask1])

#x=va_out

expr_out=Dense(8, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
expr_out_masked=tf.keras.layers.Multiply(name='expr_out')([expr_out,mask2])
aus_out=Dense(12, activation='sigmoid',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
aus_out_masked=tf.keras.layers.Multiply(name='aus_out')([aus_out,mask3])

mtlModel=tf.keras.Model(inputs=[img,mask1,mask2,mask3], outputs=[va_out_masked, expr_out_masked,aus_out_masked])

In [43]:
mtlModel.compile(optimizer=Adam(lr=1e-3), loss=[loss_va,loss_expr,loss_aus], metrics=[["mean_absolute_error"], ["accuracy"],metrics])
mtlModel.summary()

save_best_model = SaveBestModel('val_loss',False)
mtlModel.fit([X_train,masks_va_train,masks_expr_train,masks_aus_train],
             [y_va_train,y_expr_train,y_aus_train], batch_size=batch_size, epochs=(1 if TRAIN_VAL else 20), 
             verbose=1, callbacks=[save_best_model], 
             validation_data=([X_val,masks_va_val,masks_expr_val,masks_aus_val],[y_va_val,y_expr_val,y_aus_val]))
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 1416)]       0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 2)            2834        input_5[0][0]                    
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 8)            11336       input_5[0][0]                    
____________________________________________________________________________________________

In [26]:
def print_all():
    y_pred_va,y_pred_expr,y_pred_aus=mtlModel.predict([X_val,masks_va_val,masks_expr_val,masks_aus_val])
    print('\nAV')
    gt_V=y_va_val[masks_va_val==1,0]
    gt_A=y_va_val[masks_va_val==1,1]
    pred_V=y_pred_va[masks_va_val==1,0]
    pred_A=y_pred_va[masks_va_val==1,1]
    ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V,pred_A)
    print(gt_V.shape,ccc_V,ccc_A,ccc_VA)
    
    print('\nExpression')
    print(y_expr_val[masks_expr_val==1].shape)
    y_pred=np.argmax(y_pred_expr,axis=1)
    print((y_pred==y_expr_val)[masks_expr_val==1].mean())
    f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred[masks_expr_val==1], average="macro")
    print(f1_expr)
    print(metric_for_Exp(y_expr_val[masks_expr_val==1],y_pred[masks_expr_val==1]))
    
    print('\nAUs')
    new_pred = ((y_pred_aus >= 0.5) * 1)
    print(new_pred[masks_aus_val==1,:].shape)
    f1_au=np.mean([f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus.shape[1])])
    print(f1_au)
    print(f1_score_max(y_aus_val[masks_aus_val==1,:],y_pred_aus[masks_aus_val==1,:],thresh=np.arange(0.1,1,0.1)))
    
    total=ccc_VA+f1_expr+f1_au
    print('\nTotal',ccc_VA,f1_expr,f1_au,total)

In [44]:
print_all()


AV
(26876,) 0.5544011376994288 0.4413964265185361 0.4978987821089824

Expression
(15440,)
0.48555699481865283
0.4408159586324132
(0.4408159586324132, 0.48555699481865283, [0.40022913881993505, 0.3656227239621267, 0.5551684088269454, 0.2686046511627907, 0.5529940961484398, 0.5714285714285714, 0.26362221069019204, 0.5488578680203046])

AUs
(26876, 12)
0.46332223380609444


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.49661811530312255, 0.5416666666666666, array([0.53615852, 0.42304198, 0.57974368, 0.56890792, 0.73536316,
       0.71222251, 0.65327814, 0.11781089, 0.11587744, 0.27760252,
       0.87318841, 0.36622222]), array([0.5, 0.7, 0.6, 0.4, 0.4, 0.4, 0.5, 0.6, 0.6, 0.8, 0.3, 0.7]), array([0.76134841, 0.88186486, 0.82430421, 0.6254651 , 0.71055961,
       0.71011311, 0.76938533, 0.76428784, 0.94095103, 0.94887632,
       0.79163566, 0.86735377]))

Total 0.4978987821089824 0.4408159586324132 0.46332223380609444 1.40203697454749


In [28]:
print('Best weights:')
mtlModel.set_weights(best_model_weights)
print_all()

Best weights:

AV
(26876,) 0.4218102556513553 0.3570151061777144 0.38941268091453485

Expression
(15440,)
0.3701424870466321
0.2966496393771772
(0.2966496393771772, 0.3701424870466321, [0.34612162841346955, 0.335718932986337, 0.24719101123595505, 0.06359945872801083, 0.4780876494023904, 0.33641404805914976, 0.14922048997772827, 0.4168438962143769])

AUs
(26876, 12)
0.46193077632133167


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.48747842211810827, 0.5333333333333332, array([0.5322594 , 0.42471679, 0.56885418, 0.55571648, 0.73383967,
       0.70895689, 0.64713913, 0.10933941, 0.07315358, 0.26336249,
       0.8707551 , 0.36164794]), array([0.5, 0.7, 0.5, 0.4, 0.4, 0.4, 0.6, 0.6, 0.6, 0.8, 0.3, 0.6]), array([0.7904078 , 0.89607829, 0.78635214, 0.62334425, 0.70140646,
       0.7091085 , 0.78843578, 0.79632386, 0.8529171 , 0.9435928 ,
       0.78843578, 0.84145706]))

Total 0.38941268091453485 0.2966496393771772 0.46193077632133167 1.1479930966130436


In [29]:
if False:
    #mtlModel.save_weights('mtl_enet2_8.h5') #0.3837151018856935 0.3022424974391059 0.46080879636948063 1.1467663956942802
    mtlModel.save_weights('mtl_enet2_8.h5') #0.38941268091453485 0.2966496393771772 0.46193077632133167 1.1479930966130436
    #mtlModel.save_weights('mtl_enet0_vgaf.h5') #0.38155559224902114 0.2950259083661194 0.4574066636993808 1.1339881643145213
else:
    mtlModel.load_weights('mtl_enet2_8_orig.h5')
    print_all()

mobilenet
output layers only: 0.3575652288180158 0.274931498498287 0.4667221169978419 1.0992188443141446
0.35813183800284104 0.2823732083688107 0.47113929424277273 1.1116443406144245
dense layer: 0.3212823862697798 0.25187087483194115 0.46361689956738755 1.0367701606691084

enet0_7
output layers only:0.37147839426933726 0.2862848390733215 0.4561286864943848 1.1138919198370436
dense layer: 0.37387452943730165 0.2522474413566924 0.4641164043127432 1.0902383751067373
not best 0.3944477107433102 0.23835206524054334 0.4603934421520726 1.093193218135926

enet_b0_8_best_afew
features+emotions
output layers only:0.4026537613423008 0.248547670046962 0.45890535417538064 1.1101067855646434


enet_b0_8_best_vgaf
features+emotions
output layers only:0.38615587195723955 0.282608775233668 0.4545389225496568 1.1233035697405644
dense: 0.3808455198959431 0.27195008480301336 0.46871028488210437 1.1215058895810608


enet2_8
output layers only: 0.38307364079552575 0.28723802667892157 0.4593295818373552 1.1296412493118027
dense layer: 0.3966562712594687 0.281604125663917 0.45615746671167773 1.1344178636350635

features+emotions
0.3837151018856935 0.3022424974391059 0.46080879636948063 1.1467663956942802
+dense: 0.3779975030622512 0.2987745729052591 0.45822890901489305 1.1350009849824034


enet2_7
features
output layers only: 0.39352969999919685 0.2741800561672634 0.4611839520090892 1.1288937081755495
dense layer: 0.3678038228570031 0.2800059725435074 0.4564833238081867 1.104293119208697

features+emotions
output layers only: 0.38307364079552575 0.28723802667892157 0.4593295818373552 1.1296412493118027
dense layer:  0.3919721221529454 0.2821609899822945 0.460412089791669 1.1345452019269089

In [45]:
if True:
    y_pred_va,y_pred_expr,y_pred_aus=mtlModel.predict([X_val,masks_va_val,masks_expr_val,masks_aus_val])
#thresholds=np.array([0.5, 0.7, 0.5, 0.4, 0.4, 0.5, 0.5, 0.6, 0.6, 0.8, 0.3, 0.7]) #enet2
thresholds=np.array([0.5, 0.7, 0.5, 0.4, 0.4, 0.4, 0.6, 0.6, 0.6, 0.8, 0.3, 0.6]) #enet2
#thresholds=np.array([0.6, 0.7, 0.6, 0.4, 0.4, 0.5, 0.5, 0.6, 0.6, 0.7, 0.2, 0.7]) #enet0
new_pred = ((y_pred_aus >= thresholds) * 1)
f1_au=np.mean([f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus.shape[1])])
print(f1_au)


0.49562541673982846


### Save test results 

In [ ]:
if False:
    train_val_videos=set()
    for filename in ['train_set.txt', 'validation_set.txt']:
        with open(os.path.join(DATA_DIR,'Third ABAW Annotations/MTL_Challenge/'+filename)) as f:
            mtl_lines = f.read().splitlines()
        for line in mtl_lines[1:]:
            splitted_line=line.split(',')
            imagename=splitted_line[0]
            videoname=imagename.split('/')[0]
            train_val_videos.add(videoname)
        print(len(train_val_videos))
    
    data_dir=os.path.join(DATA_DIR,'faces')
    test_videos={}
    for videoname in os.listdir(data_dir):
        if videoname not in train_val_videos:
            X,indices,filenames=[],[],[]
            images=[img_name for img_name in os.listdir(os.path.join(data_dir,videoname)) if img_name.lower().endswith('.jpg')]
            for img_name in sorted(images, key=compare_filenames):
                k=videoname+'/'+img_name
                if k in filename2featuresAll:
                    X.append(np.concatenate((filename2featuresAll[k][0],filename2featuresAll[k][1])))
                    indices.append(int(img_name[:-4]))
                    filenames.append(k)
            test_videos[videoname]=(np.array(X),indices,filenames)
    print(len(test_videos))
    
    resdir=os.path.join(DATA_DIR,'test_results/MTL')
    header = 'image,valence,arousal,expression,aus\n'
    with open(os.path.join(resdir,'MTL_predictions.txt'), 'w') as f:
        f.write(header)
        for videoname,(X,indices,filenames) in test_videos.items():
            mask_ones=np.ones(len(X))
            y_pred_va,y_pred_expr,y_pred_aus=mtlModel.predict([X,mask_ones,mask_ones,mask_ones])
            y_expr=np.argmin(y_pred_expr,axis=1)
            y_aus=(y_pred_aus>=thresholds)*1

            for filename, va,expr,aus in zip(filenames,y_pred_va,y_expr,y_aus):
                s=','.join([filename,str(va[0]),str(va[1]),str(expr),*map(str,aus)])+'\n'
                f.write(s)

In [33]:
with open(os.path.join(DATA_DIR,'Multi_Task_Learning_Challenge_test_set_release.txt'),'r') as f:
    test_set_videos=f.read().splitlines()
print(len(test_set_videos),test_set_videos[:5])

51246 ['image,valence,arousal,expression,aus', 'video45_1/00001.jpg', 'video45_1/00008.jpg', 'video45_1/00017.jpg', 'video45_1/00021.jpg']


In [46]:
resdir=os.path.join(DATA_DIR,'test_results/MTL/4_e2')
if not os.path.exists(resdir):
    os.makedirs(resdir)
header = 'image,valence,arousal,expression,aus\n'
with open(os.path.join(resdir,'MTL_predictions.txt'), 'w') as f:
    f.write(header)
    X,filenames=[],[]
    mask_ones=np.ones(batch_size)
    
    for k in test_set_videos[1:]:
        if k in filename2featuresAll:
            X.append(np.concatenate((filename2featuresAll[k][0],filename2featuresAll[k][1])))
            filenames.append(k)
            
            if len(X)==batch_size:
                y_pred_va,y_pred_expr,y_pred_aus=mtlModel.predict([np.array(X),mask_ones,mask_ones,mask_ones])
                y_expr=np.argmin(y_pred_expr,axis=1)
                y_aus=(y_pred_aus>=thresholds)*1

                for filename, va,expr,aus in zip(filenames,y_pred_va,y_expr,y_aus):
                    s=','.join([filename,str(va[0]),str(va[1]),str(expr),*map(str,aus)])+'\n'
                    f.write(s)
                X,filenames=[],[]
        else:
            print(k, 'not found')
    if len(X)>0:
        mask_ones=np.ones(len(X))
        y_pred_va,y_pred_expr,y_pred_aus=mtlModel.predict([np.array(X),mask_ones,mask_ones,mask_ones])
        y_expr=np.argmin(y_pred_expr,axis=1)
        y_aus=(y_pred_aus>=thresholds)*1

        for filename, va,expr,aus in zip(filenames,y_pred_va,y_expr,y_aus):
            s=','.join([filename,str(va[0]),str(va[1]),str(expr),*map(str,aus)])+'\n'
            f.write(s)
        X,filenames=[],[]